In [2]:
# import numpy as np
# import pandas as pd

In [3]:
import openai
import os

In [4]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [5]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")



In [6]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [7]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [8]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

from langchain_community.document_loaders import PyPDFLoader

file_path = "/home/nouman-aziz/Downloads/Grid Needs Assessment SCE 2024.pdf"

loader = PyPDFLoader(file_path)

pages = []
async for page in loader.alazy_load():
    pages.append(page)

docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

/home/nouman-aziz/demo_bot/venv/lib/python3.12/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
for i in range(10):
    user_input = input("Enter your question here: ")
    for step in graph.stream(
        {"messages": [{"role": "user", "content": user_input}]},
        stream_mode="values",
        config=config,
    ):
        step["messages"][-1].pretty_print()

In [26]:
response

'I am an AI language model developed by OpenAI, and I don\'t have a personal name. You can simply refer to me as "Assistant." How can I assist you today?'

In [23]:
step["messages"][-1]

AIMessage(content="Please provide the document or the specific content you're referring to, and I'll assist you in identifying the factors included in it.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 136, 'total_tokens': 162, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_27322b4e16', 'id': 'chatcmpl-BFRrNqAxccRPX0XPIlmSTiZLcWaKV', 'finish_reason': 'stop', 'logprobs': None}, id='run-b821ce70-baf8-4093-8eec-7f5ec3bdc38f-0', usage_metadata={'input_tokens': 136, 'output_tokens': 26, 'total_tokens': 162, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [24]:
step["messages"][-1].content

"Please provide the document or the specific content you're referring to, and I'll assist you in identifying the factors included in it."

In [17]:
for i in range(10):
    user_input = input("Enter your question here: ")
    if user_input.lower() =="quit":
       break
    response = graph.invoke({"question": user_input})
    print(response["answer"])